[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1_wJvZr5c8mi-59PSsDlPszkgD2hUIo0t?usp=sharing)


In [ ]:
# @title Install dependencies
%%capture

!uv pip cache purge

!uv pip install --system unsloth wandb ray # bitsandbytes triton
# Get the latest nightly Unsloth
# %pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

# Install Flash Attention 2 for softcapping support
import torch

if torch.cuda.get_device_capability()[0] >= 8:
    !uv pip install --system --no-deps packaging ninja einops "flash-attn>=2.6.3"

### Get current GPU and memory stats


In [3]:
#!nvidia-smi

In [4]:
import psutil

# Get the memory usage
memory_info = psutil.virtual_memory()
print(f"Total Memory: {memory_info.total / (1024**3):.2f} GB")
print(f"Available Memory: {memory_info.available / (1024**3):.2f} GB")
print(f"Used Memory: {memory_info.used / (1024**3):.2f} GB")
print(f"Memory Usage: {memory_info.percent}%")

Total Memory: 12.67 GB
Available Memory: 11.05 GB
Used Memory: 1.31 GB
Memory Usage: 12.8%


### Load model


In [ ]:
from unsloth import FastLanguageModel

max_seq_length = 1024
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/gemma-2-9b",
    max_seq_length=max_seq_length,
    dtype=None,
    load_in_4bit=True,
    # local_files_only=True,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.12.4: Fast Gemma2 patching. Transformers:4.46.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/6.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

### Inference


In [ ]:
import sys

from huggingface_hub import login


def is_colab():
    return "google.colab" in sys.modules


if is_colab():
    from google.colab import userdata

    HF_TOKEN = userdata.get("HF_TOKEN")
    WB_API_KEY = userdata.get("WB_API_KEY")

    if HF_TOKEN is None:
        raise KeyError("HF_TOKEN not found in Google Colab userdata.")

else:
    import os

    from dotenv import load_dotenv

    load_dotenv()

    try:
        HF_TOKEN = os.getenv("HF_TOKEN")
        WB_API_KEY = os.getenv("WB_API_KEY")
    except KeyError:
        raise KeyError("HF_TOKEN not found in environment variables.")


login(token=HF_TOKEN, add_to_git_credential=True)
print("Login to HF hub successfully!")

Login to HF hub successfully!


<a name="Data"></a>

### Data Prep

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!


In [ ]:
from datasets import load_dataset

mnemonic_prompt = """
Below is an English term. Your task is to generate a imageable (elicit strong mental image) and memorable mnemonic to help remember its meaning.

### Term:
{}

### Mnemonic:
{}"""

EOS_TOKEN = tokenizer.eos_token  # End-of-sequence token


# Formatting function
def formatting_prompts_func(examples):
    terms = examples["term"]
    mnemonics = examples["mnemonic"]
    texts = []
    for term, mnemonic in zip(terms, mnemonics, strict=False):
        text = mnemonic_prompt.format(term, mnemonic) + EOS_TOKEN
        texts.append(text)
    return {
        "text": texts,
    }


# Load and process the dataset
dataset = load_dataset(
    "chiffonng/en-vocab-mnemonics",
    split={"train": "train", "test": "test"},
).map(formatting_prompts_func, batched=True)

train_dataset = dataset["train"]
eval_dataset = dataset["test"]

README.md:   0%|          | 0.00/2.99k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/84.4k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/23.7k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1053 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/264 [00:00<?, ? examples/s]

Map:   0%|          | 0/1053 [00:00<?, ? examples/s]

Map:   0%|          | 0/264 [00:00<?, ? examples/s]

<a name="Train"></a>

### Train the model (Supervised fine-tuning, SFT)

Use Huggingface TRL's `SFTTrainer` for supervised finetuning! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer).


In [8]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
6.676 GB of memory reserved.


#### LoRA configuration


In [9]:
model = FastLanguageModel.get_peft_model(
    model,
    r=8,  # Rank > 0. Suggested 8, 16, 32, 64, 128. Higher ranks can capture more complex patterns but may increase the risk of overfitting and require more computational resources.
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=16,  # = alpha/\sqrt{r} is a scaling factor for rank-stabilized LoRA. Should be >= 1.
    lora_dropout=0,  # Slight dropout to prevent overfitting
    bias="none",  # Supports any, but = "none" is optimized
    # "unsloth" uses 30% less VRAM, fits 2x larger batch sizes
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=True,  # Rank stabilized LoRA
    loftq_config=None,  # And LoftQ
)

Unsloth 2024.12.4 patched 42 layers with 42 QKV layers, 42 O layers and 42 MLP layers.


##


TODO here:

- Hyperparameter tune
  - `lr_scheduler_type`: restart intervals and decay parameters


#### Logging

- HuggingFace transformers and wandb integration: [guide](https://docs.wandb.ai/guides/integrations/huggingface/)


In [ ]:
# TODO: Switch type to ExplicitEnum
def init_report(report_service: str = "wandb"):
    """Initialize chosen service to report metrics and progress during training.

    Args:
        report_service (str): "none" or "wandb".

    Returns:
        report_service (str): "none" or "wandb".
    """
    if report_service not in ["wandb"]:
        report_service = "none"

    if report_service == "wandb":
        import wandb

        wandb.login(key=WB_API_KEY)
        wandb.init(project="gemma2-9b-it-mnemonics", log_model=True)
    return report_service

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


env: WANDB_PROJECT="gemma2-9b-it-mnemonics"
env: WANDB_LOG_MODEL="checkpoint"


wandb: Currently logged in as: chiffonng (chiffonng-minerva-university). Use `wandb login --relogin` to force relogin


In [ ]:
from transformers import EarlyStoppingCallback, TrainingArguments
from trl import SFTTrainer
from unsloth import is_bfloat16_supported


def train_func():
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]

    train_args = TrainingArguments(
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        gradient_accumulation_steps=4,  # batch size of 16
        warmup_steps=10,
        num_train_epochs=4,
        learning_rate=2e-5,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="cosine_with_restarts",  # Cosine Annealing with Restarts
        seed=3407,
        # Save strategy
        output_dir="./outputs",
        save_strategy="steps",
        save_steps=20,
        load_best_model_at_end=True,
        save_total_limit=5,
        # Eval strategy
        metric_for_best_model="eval_loss",
        eval_strategy="steps",  # Enable evaluation during training
        eval_steps=20,  # Evaluate every 20 steps
        # Logging
        logging_steps=10,
        report_to=REPORT_SERVICE,
    )

    trainer = SFTTrainer(
        model=model,
        tokenizer=tokenizer,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        dataset_text_field="text",
        max_seq_length=max_seq_length,
        dataset_num_proc=2,
        packing=True,
        args=train_args,
        callbacks=callbacks,
    )

    trainer_stats = trainer.train(
        # resume_from_checkpoint=True
    )


train_func()

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Received unrecognized `WANDB_LOG_MODEL` setting value="checkpoint"; so disabling `WANDB_LOG_MODEL`
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 69 | Num Epochs = 4
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 4
\        /    Total batch size = 16 | Total steps = 16
 "-____-"     Number of trainable parameters = 27,009,024
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

### Script to run


In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime'] / 60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

<a name="Inference"></a>

### Inference

Let's run the model! You can change the instruction and input - leave the output blank!


In [ ]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
    [
        mnemonic_prompt.format(
            "preposterous",  # term
            "",  # mnemonic
        )
    ],
    return_tensors="pt",
).to("cuda")

from transformers import TextStreamer

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=128, use_cache=True)

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.


In [ ]:
model.push_to_hub_merged(
    "gemma2-9b-mnemonics-sft",
    tokenizer,
    save_method="merged_16bit",
    token=HF_TOKEN,
    safe_serialization=None,  # save to .safetensors
)

model.push_to_hub_merged(
    "gemma2-9b-mnemonics-sft-4bit",
    tokenizer,
    save_method="merged_4bit",
    token=HF_TOKEN,
    safe_serialization=None,  # save to .safetensors
)